In [2]:
import datetime
import importlib
from types import SimpleNamespace

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras

np.random.seed(0)
sns.set()

In [3]:
%ls
# bring the data file csv for

buildingDB.csv  sample_data/


In [4]:
#keras extra

import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
# from IPython.display import clear_output


def plot_history(history):
    # Plot training & validation loss values
    plt.plot(history["loss"])
    plt.plot(history["val_loss"])
    plt.title("Model loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Test"], loc="upper left")
    plt.show()


class PlotLossesBatch(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []

        self.fig = plt.figure()

        self.logs = []

    def on_train_batch_end(self, batch, logs=None):
        if batch == 0 or batch % 50 != 0:
            return

        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get("loss"))
        self.val_losses.append(logs.get("val_loss"))
        self.i += 1

        # clear_output(wait=True)

        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show()

    def on_epoch_end(self, epoch, logs={}):

        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get("loss"))
        self.val_losses.append(logs.get("val_loss"))
        self.i += 1

        # clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show()


class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []

        self.fig = plt.figure()

        self.logs = []

    def on_epoch_end(self, epoch, logs={}):

        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get("loss"))
        self.val_losses.append(logs.get("val_loss"))
        self.i += 1

        # clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show()


In [5]:
# datamunge 
import numpy as np
from sklearn.preprocessing import MinMaxScaler


def rolling_window(series, window_size):
    """
    Transforms an array of series into an array of sliding window arrays. If
    the passed in series is a matrix, each column will be transformed into an
    array of sliding windows.
    """
    return np.array(
        [
            series[i : (i + window_size)]
            for i in range(0, series.shape[0] - window_size + 1)
        ]
    )


def multi_shuffle(inputs):
    """
    Shuffles arrays of the same length in lockstep.
    """
    
    indices = np.arange(inputs[0].shape[0])
    np.random.shuffle(indices)

    return list(map(lambda x: x[indices], inputs))


def dual_shuffle(x, y):
    """
    Shuffles two arrays of the same length in the same positions.
    """
    results = multi_shuffle([x, y])
    return results[0], results[1]


def to_timeseries_input(series, lookback, predictions, output_col=0):
    inputs = rolling_window(series[:-predictions, :], lookback)
    outputs = rolling_window(series[lookback:, output_col], predictions)

    return inputs, outputs


def to_timeseries_by_partition(series, partition, lookback, predictions, output_col=0):
    parts = series[partition].unique()

    series_x = []
    series_y = []

    for part in parts:
        ds_partition = (
            series[series[partition] == part].drop([partition], axis=1).values
        )
        ts_x, ts_y = to_timeseries_input(
            ds_partition, lookback, predictions, output_col
        )
        series_x.append(ts_x)
        series_y.append(ts_y)

    return np.concatenate(series_x), np.concatenate(series_y)


def normalize_dataset(df, train_split_index):
    """
    Normalizes the given dataframe, using only the train data. The test train
    split is decided by the passed in split index.

    Args:
        df (dataframe): a pandas dataframe to normalize
        train_split_index (datetime): a datetime object by which to split the
            dataframe
    """
    scaler = MinMaxScaler()
    scaler.fit(df.iloc[0:train_split_index])
    return (scaler, scaler.transform(df))


def rescale(scaler, in_y, orig_columns=5):
    req_shape = np.dstack([in_y for i in range(0, orig_columns)]).reshape(
        in_y.shape[0], orig_columns
    )
    return scaler.inverse_transform(req_shape)[:, 0]


def find_split_index(df, split_date):
    """
    Finds the index of the split point of the dataframe were it to be
    split by the passed in date time.

    Args:
        df (dataframe): a pandas dataframe to split by date
        split_date (datetime): a datetime object by which to split the
            dataframe
    """
    return len(df[df.index < split_date])


def lag_normalize_split(df, split_date, lookback=12, num_predictions=12, output_col=0):
    train_split_index = find_split_index(df, split_date)

    scaler, scaled = normalize_dataset(df, train_split_index)

    train = scaled[0:train_split_index]
    test = scaled[train_split_index:]

    train_x, train_y = to_timeseries_input(
        train, lookback, num_predictions, output_col=output_col
    )
    # train_x, train_y = dual_shuffle(train_x, train_y)

    test_x, test_y = to_timeseries_input(
        test, lookback, num_predictions, output_col=output_col
    )

    return (scaler, train_x, test_x, train_y, test_y)


def remove_last_dim(arr):
    """
    Reshapes the given array to remove the last dimension (this makes
    the assumption that the last dimension is of shape 1).
    """
    return arr.reshape(arr.shape[0], arr.shape[1])


def prepare_inputs_by_partition(
    df,
    partition_col,
    split_date,
    categorical_cols=None,
    output_col=0,
    lookback=12,
    num_predictions=12,
):
    """
    Lags, splits and normalizes a dataframe based around a partition.
    """
    partitions = df[partition_col].unique()
    scalers = {}
    train_x = None
    test_x = None
    train_y = None
    test_y = None
    testset_by_partition = {}
    for partition in partitions:
        df_part = df.loc[df[partition_col] == partition].copy()

        if categorical_cols is None:
            df_cat_train = None
            df_cat_test = None
        else:
            train_split_index = find_split_index(df_part, split_date)
            df_cat_train = df_part.iloc[
                :train_split_index, categorical_cols
            ].values.astype(np.float32)
            df_cat_test = df_part.iloc[
                train_split_index:, categorical_cols
            ].values.astype(np.float32)
            df_part.drop(df_part.columns[categorical_cols], axis=1, inplace=True)

        df_part.drop([partition_col], axis=1, inplace=True)

        scaler, tr_x, te_x, tr_y, te_y = lag_normalize_split(
            df_part,
            split_date,
            output_col=output_col,
            lookback=lookback,
            num_predictions=num_predictions,
        )
        scalers[partition] = scaler
        
        testset_by_partition[partition] = {
            "test_x": te_x
            if df_cat_test is None
            else [te_x, df_cat_test[0 : len(te_x)]],
            "test_y": remove_last_dim(te_y),
        }

        if train_x is None:
            train_x = tr_x
            test_x = te_x
            train_y = tr_y
            test_y = te_y
            if not df_cat_train is None:
                train_x_cat = df_cat_train[: len(tr_x)]
                test_x_cat = df_cat_test[: len(te_x)]
        else:
            train_x = np.concatenate((train_x, tr_x))
            test_x = np.concatenate((test_x, te_x))
            train_y = np.concatenate((train_y, tr_y))
            test_y = np.concatenate((test_y, te_y))
            if not df_cat_train is None:
                train_x_cat = np.concatenate((train_x_cat, df_cat_train[: len(tr_x)]))
                test_x_cat = np.concatenate((test_x_cat, df_cat_test[: len(te_x)]))

    return (
        scalers,
        train_x if df_cat_train is None else [train_x, train_x_cat],
        test_x if df_cat_test is None else [test_x, test_x_cat],
        remove_last_dim(train_y),
        remove_last_dim(test_y),
        testset_by_partition,
    )


def shuffle_train_sets(train_x, train_y):
    if isinstance(train_x, list):
        shuffled = multi_shuffle([train_x[0], train_x[1], train_y])
        return [shuffled[0], shuffled[1]], shuffled[2]

    return dual_shuffle(train_x, train_y)


In [6]:
# date utils.py 
import pandas as pd


def make_datetime_index(start_date, end_date=None, freq="5min", periods=None):
    return pd.date_range(start=start_date, end=end_date, freq=freq, periods=periods)


def make_daterange(start_date, end_date=None, freq="5min", periods=None):
    return make_datetime_index(
        start_date, end_date, freq, periods=periods
    ).to_pydatetime()



In [7]:
#experiments

import numpy as np
from sklearn.metrics import mean_absolute_error


ALL_ROOMS = ["babbage", "babyfoot", "jacquard"]


def print_losses(history):
    print("{:<25} {:<10}".format("Loss Name", "Losses"))
    print("")
    for key, item in history.history.items():
        print("{:<25}".format(key), end="")
        for val in item:
            print("{:.5f} ".format(val), end="")
        print("")


def rescaled_mae(scaler, preds, true_vals, orig_columns=5):
    """
    Calculates the MAE after rescaling the data.
    """
    preds_rs = rescale(scaler, preds, orig_columns)
    true_vals_rs = rescale(scaler, true_vals, orig_columns)
    return mean_absolute_error(true_vals_rs, preds_rs)


def print_mae(preds, true_vals):
    """ Prints the MAE for the given prediction and truth values """
    print(f"MAE {mean_absolute_error(true_vals, preds)}")


def print_rescaled_mae(scaler, preds, true_vals, orig_columns=5):
    print(f"MAE rescaled {rescaled_mae(scaler, preds, true_vals, orig_columns)}")


def make_predictions(model, test_x, test_y):
    preds = model.predict(test_x).flatten()
    true_vals = test_y.flatten()
    return preds, true_vals


def run_predictions(model, in_test_x, in_test_y, scaler=None):
    preds, true_vals = make_predictions(model, in_test_x, in_test_y)

    print_mae(preds, true_vals)
    if not scaler is None:
        print_rescaled_mae(scaler, preds, true_vals, in_test_x.shape[2])

    return (preds, true_vals)


def make_bywindow_predictions(model, test_x, test_y):
    if isinstance(test_x, list):
        #lookback = test_x[0].shape[1]
        predictions = test_y.shape[1]
        test_set = [test_x[0][0::predictions], test_x[1][0::predictions]]
    else:
        #lookback = test_x.shape[1]
        predictions = test_y.shape[1]
        test_set = test_x[0::predictions]

    preds = model.predict(test_set).flatten()
    true_vals = test_y[0::predictions].flatten()

    return preds, true_vals


def make_bywindow_multi_predictions(model, test_x, test_sets):
    """
    Using the model and the given test sets, makes predictions based on the window
    used in the train set and joins the predictions together to make one continuous
    prediction. This multi prediction version is used for models that have multiple
    output predictions.
    """
    if isinstance(test_x, list):
        lookback = test_x[0].shape[1]
        test_set = [test_x[0][0::lookback], test_x[1][0::lookback]]
    else:
        lookback = test_x.shape[1]
        test_set = test_x[0::lookback]

    preds = model.predict(test_set)

    preds = [x.flatten() for x in preds]
    truth_sets = [x[0::lookback].flatten() for x in test_sets]

    return preds, truth_sets


def plot_preds_v_truth(preds, true_vals, truth_name="temperature", chart_index=None):
    pandas_2line_plot_from_data(
        true_vals, preds, truth_name, "predictions", chart_index
    )


def chart_index_for_room(df, split_date, room, num_predictions=12):
    test_set = df.loc[df.index >= split_date]
    return test_set[test_set.room == room].iloc[:-num_predictions].index


def prediction_chart_for_room(
    model,
    room,
    room_testsets,
    truth_name="temperature",
    chart_index=None,
    scaler=None,
    orig_columns=5,
):
    in_test_x = room_testsets[room]["test_x"]
    in_test_y = room_testsets[room]["test_y"]

    preds, true_vals = make_bywindow_predictions(model, in_test_x, in_test_y)

    if not scaler is None:
        preds = rescale(scaler, preds, orig_columns)
        true_vals = rescale(scaler, true_vals, orig_columns)

    plot_preds_v_truth(
        preds, true_vals, truth_name=f"{truth_name} {room}", chart_index=chart_index
    )


def prediction_chart_for_partition(
    model,
    partition,
    testsets,
    truth_name="temperature",
    chart_index=None,
    scaler=None,
    orig_columns=5,
):
    in_test_x = testsets[partition]["test_x"]
    in_test_y = testsets[partition]["test_y"]

    preds, true_vals = make_bywindow_predictions(model, in_test_x, in_test_y)

    if not scaler is None:
        preds = rescale(scaler, preds, orig_columns)
        true_vals = rescale(scaler, true_vals, orig_columns)

    plot_preds_v_truth(
        preds,
        true_vals,
        truth_name=f"{truth_name} {partition}",
        chart_index=chart_index,
    )


def prediction_charts_by_room(
    model,
    room_testsets,
    truth_name="temperature",
    chart_index=None,
    rooms=None,
    scalers=None,
    orig_columns=5,
):
    if rooms is None:
        rooms = ALL_ROOMS

    for room in rooms:
        if scalers is None:
            scaler = None
        else:
            scaler = scalers[room]

        prediction_chart_for_room(
            model,
            room,
            room_testsets,
            truth_name=truth_name,
            chart_index=chart_index,
            scaler=scaler,
            orig_columns=orig_columns,
        )


def run_predictions_for_rooms(model, room_testsets, scalers, rooms=None):
    if rooms is None:
        rooms = ALL_ROOMS

    for room in rooms:
        in_test_x = room_testsets[room]["test_x"]
        in_test_y = room_testsets[room]["test_y"]

        scaler = scalers[room]

        print(f"Predictions for {room}")
        run_predictions(model, in_test_x, in_test_y, scaler)
        print("")


def replace_column_in_input(pred_vals, in_vals, col):
    return np.dstack((np.delete(in_vals, col, axis=2), pred_vals))


def get_slice_of_testset(test_x, start_index=0, end_index=1):
    if isinstance(test_x, list):
        return list(map(lambda x: x[start_index:end_index], test_x))

    return test_x[start_index:end_index]


def make_stepthrough_predictions(model, test_x, test_y, output_col=0, window_size=12):
    test_set = get_slice_of_testset(test_x)

    preds = model.predict(test_set).flatten()

    if isinstance(test_x, list):
        test_length = len(test_x[0])
    else:
        test_length = len(test_x)

    for i in range(window_size, test_length, window_size):
        last_prediction = preds[-window_size:]
        input_slice = get_slice_of_testset(test_x, i, i + 1)
        if isinstance(test_x, list):
            new_input = input_slice
            new_input[0] = replace_column_in_input(
                last_prediction, input_slice[0], output_col
            )
        else:
            new_input = replace_column_in_input(
                last_prediction, input_slice, output_col
            )
        preds = np.append(preds, model.predict(new_input).flatten())

    true_vals = test_y.flatten()[0::window_size]
    return preds, true_vals


def stepthrough_predictions_for_daynum(
    model, test_x, test_y, daynum=0, dayindex=0, dayspan=1, output_col=0, window_size=12
):
    slice_start = 288 * daynum + dayindex
    slice_end = slice_start + dayspan * 288
    index_slice = slice(slice_start, slice_end)

    if isinstance(test_x, list):
        test_slice_x = list(map(lambda x: x[index_slice], test_x))
    else:
        test_slice_x = test_x[index_slice]

    test_slice_y = test_y[index_slice]

    return make_stepthrough_predictions(
        model, test_slice_x, test_slice_y, output_col, window_size
    )


In [8]:
#generators

import numpy as np


def multi_gen(generators, shuffle=False):
    """
    Generator that combines multiple other generators to return samples. Will
    either return a value from each generator in succession or randomly
    depending on the value of the shuffle parameter.
    """
    i = -1

    while 1:
        i = np.random.randint(0, 3) if shuffle else (i + 1) % len(generators)
        gen = generators[i]
        sample = next(gen)
        yield sample


def ts_generator(
    data,
    lookback,
    target_col=0,
    n_outputs=1,
    step=1,
    min_index=0,
    max_index=None,
    delay=0,
    shuffle=False,
    batch_size=16,
):
    """
    Generator that creates 3d time series shaped data for use in RNN layers
    or similar

    Args:
        data (array): an indexable matrix of timeseries data
        lookback (int): how many timesteps back the input data should go
        delay (int): how many steps into the future the target should be
        min_index (int): point in data at which to start
        max_index (int): point in data at which to finish
        shuffle (boolean): whether to shuffle the samples
        batch_size (int): the number of samples per batch
        step (int): the period in timesteps at which to sample the data
    """
    if max_index is None:
        max_index = len(data) - delay - n_outputs

    i = min_index + lookback

    # if shuffle:
    #    np.random.shuffle(data)

    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows), n_outputs))

        for j, _ in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            target_start = rows[j] + delay
            target_end = target_start + n_outputs
            targets[j] = data[target_start:target_end, target_col]

        if n_outputs == 1:
            targets = targets.reshape(targets.shape[:-1])

        yield samples, targets, [None]


def ts_seasonal_generator(
    data,
    target_col=0,
    block_size=24,
    n_outputs=12,
    step=1,
    min_index=0,
    max_index=None,
    delay=0,
    shuffle=False,
    batch_size=16,
    freq=5,
):
    """
    Generator that creates 3d time series shaped data for use in RNN layers
    or similar

    Args:
        data (array): an indexable matrix of timeseries data
        lookback (int): how many timesteps back the input data should go
        delay (int): how many steps into the future the target should be
        min_index (int): point in data at which to start
        max_index (int): point in data at which to finish
        shuffle (boolean): whether to shuffle the samples
        batch_size (int): the number of samples per batch
        step (int): the period in timesteps at which to sample the data
    """
    half_sample = block_size // 2
    lookback = (60 // freq) * 24 * 7 + half_sample
    lookback_d = (60 // freq) * 24 + half_sample

    if max_index is None:
        max_index = len(data) - delay - n_outputs

    i = min_index + lookback

    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), block_size // step, data.shape[-1] * 3))
        targets = np.zeros((len(rows), n_outputs))

        for j, _ in enumerate(rows):
            indices1 = range(rows[j] - block_size, rows[j], step)
            indices2 = range(rows[j] - lookback, rows[j] - lookback + block_size, step)
            indices3 = range(rows[j] - lookback_d, rows[j] - lookback_d + block_size, step)
            data1 = data[indices1]
            data2 = data[indices2]
            data3 = data[indices3]
            all_data = np.hstack((data1, data2, data3))

            #print(samples.shape)
            #print(data1.shape, data2.shape, data3.shape)
            #print(all_data.shape)
            samples[j] = all_data

            target_start = rows[j] + delay
            target_end = target_start + n_outputs
            targets[j] = data[target_start:target_end, target_col]

        if n_outputs == 1:
            targets = targets.reshape(targets.shape[:-1])

        yield samples, targets, [None]


In [ ]:
!pip install keras-tcn

In [10]:
#keras model

import datetime
import os

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import callbacks as kcb
from tensorflow.keras.layers import (
    GRU,
    LSTM,
    Conv1D,
    Dense,
    Flatten,
    MaxPooling1D,
    RepeatVector,
)
from tensorflow.keras.models import Sequential
from tcn import TCN

from tqdm.keras import TqdmCallback


def get_tb_logdir():
    return os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))


def learning_rate_finder_train(model, train_x, train_y, loss="mae", epochs=10):
    lr_schedule = keras.callbacks.LearningRateScheduler(
        lambda epoch: 1e-8 * 10 ** (epoch / 20)
    )
    optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
    model.compile(loss=loss, optimizer=optimizer)
    history = model.fit(
        train_x,
        train_y,
        callbacks=[lr_schedule, TqdmCallback(verbose=1)],
        epochs=epochs,
        verbose=0,
    )
    plot_learning_rate_history(history)
    return history


def train_model(
    model,
    train_x,
    train_y,
    test_x,
    test_y,
    use_tensorboard=False,
    live_valchart=False,
    epochs=10,
    batch_size=16,
    verbose=0,
    early_stopping_patience=4,
):
    callbacks = [TqdmCallback(verbose=1)]

    if early_stopping_patience > 0:
        callbacks.append(
            keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
        )

    if use_tensorboard:
        logdir = get_tb_logdir()
        tensorboard_callback = kcb.TensorBoard(
            logdir, histogram_freq=1, profile_batch=0
        )
        callbacks.append(tensorboard_callback)

    if live_valchart:
        plot_losses = PlotLosses()
        callbacks.append(plot_losses)

    return model.fit(
        train_x,
        train_y,
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose,
        validation_data=(test_x, test_y),
        callbacks=callbacks,
    )


def get_rnn_shapes(x, y):
    return (
        x.shape[1],
        x.shape[2],
        y.shape[1],
    )


def build_model(
    model_func,
    train_x,
    train_y,
    test_x,
    test_y,
    epochs=15,
    batch_size=16,
    use_tensorboard=False,
):
    n_timesteps, n_features, n_outputs = get_rnn_shapes(train_x, train_y)

    verbose = 1
    input_shape = (n_timesteps, n_features)

    # define model
    model = model_func(input_shape, n_outputs)

    out_shape = model.layers[-1].output_shape[1:]
    train_y = train_y.reshape((train_y.shape[0],) + out_shape)
    test_y = test_y.reshape((test_y.shape[0],) + out_shape)

    return train_model(
        model,
        train_x,
        train_y,
        test_x,
        test_y,
        use_tensorboard=use_tensorboard,
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose,
    )


def model_encoderdecoder(input_shape, n_outputs, activation="relu", loss="mae"):
    model = Sequential()
    model.add(LSTM(100, activation=activation, input_shape=input_shape))
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(100, activation=activation, return_sequences=True))
    model.add(Dense(100, activation=activation))
    model.add(Dense(1, activation=activation))
    model.compile(loss=loss, optimizer="adam")
    return model


def model_encoderdecoder_tanh(input_shape, n_outputs, loss="mae"):
    return model_encoderdecoder(input_shape, n_outputs, activation="tanh", loss=loss)


def model_convnet(input_shape, n_outputs, activation="relu", loss="mae"):
    model = Sequential()
    model.add(
        Conv1D(
            filters=32, kernel_size=3, activation=activation, input_shape=input_shape
        )
    )
    model.add(Conv1D(filters=32, kernel_size=3, activation=activation))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=16, kernel_size=3, activation=activation))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation=activation))
    model.add(Dense(n_outputs))
    model.compile(loss=loss, optimizer="adam")
    return model


def model_simple_lstm(input_shape, n_outputs, loss="mae"):
    model = Sequential()
    model.add(LSTM(50, input_shape=input_shape))
    model.add(Dense(n_outputs))
    model.compile(loss=loss, optimizer="adam")
    return model


def model_simple_gru(input_shape, n_outputs, loss="mae", optimizer="adam"):
    model = Sequential()
    model.add(GRU(32, input_shape=input_shape))
    model.add(Dense(n_outputs))
    model.compile(loss=loss, optimizer=optimizer)
    return model


def model_simple_tcn(input_shape, n_outputs, loss="mae", optimizer="adam"):
    i = keras.layers.Input(shape=input_shape)

    x = TCN(return_sequences=False)(i)
    x = keras.layers.Dense(n_outputs)(x)

    model = keras.Model(i, x)
    model.compile(optimizer=optimizer, loss=loss)
    return model


def model_gru_conv(input_shape, n_outputs, loss="mae", optimizer="adam"):
    inp = keras.layers.Input(shape=input_shape)
    x = keras.layers.GRU(32, return_sequences=True)(inp)
    x = keras.layers.AveragePooling1D(2)(x)
    x = keras.layers.Conv1D(32, 3, activation="relu", padding="same", name="f_extract")(
        x
    )
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(16, activation="relu")(x)
    out = keras.layers.Dense(n_outputs)(x)

    model = keras.models.Model(inp, out)
    model.compile(optimizer=optimizer, loss=loss)

    return model


def model_conditioned_gru(n_outputs, train_x, loss="mae", optimizer="adam"):
    model = ConditionedGRU(num_outputs=n_outputs)
    # model.call(train_x)
    model.compile(loss=loss, optimizer=optimizer)
    return model


def model_conditioned_lstm(n_outputs, train_x, loss="mae", optimizer="adam"):
    model = ConditionedLSTM(num_outputs=n_outputs)
    model.call(train_x)
    model.compile(loss=loss, optimizer=optimizer)
    return model


class ConditionedLSTM(keras.Model):
    def __init__(self, num_outputs):
        super(ConditionedLSTM, self).__init__()
        self.cond = ConditionalRNNLayer(
            32, cell="LSTM", return_sequences=True, dtype=tf.float32
        )
        self.lstm2 = keras.layers.LSTM(24, return_sequences=False)
        self.out = keras.layers.Dense(num_outputs)

    def call(self, inputs, **kwargs):
        x = self.cond(inputs)
        x = self.lstm2(x)
        x = self.out(x)
        return x


class ConditionedGRU(keras.Model):
    def __init__(self, num_outputs):
        super(ConditionedGRU, self).__init__()
        self.cond = ConditionalRNNLayer(
            32, cell="GRU", return_sequences=False, dtype=tf.float32
        )
        self.out = keras.layers.Dense(num_outputs)

    def call(self, inputs, **kwargs):
        o = self.cond(inputs)
        o = self.out(o)
        return o


class ConditionalRNNLayer(keras.layers.Layer):
    # Arguments to the RNN like return_sequences, return_state...
    def __init__(self, units, cell=keras.layers.LSTMCell, *args, **kwargs):
        """
        Conditional RNN. Conditions time series on categorical data.
        :param units: int, The number of units in the RNN Cell
        :param cell: string, cell class or object (pre-instantiated). In the case of string, 'GRU',
        'LSTM' and 'RNN' are supported.
        :param args: Any parameters of the keras.layers.RNN class, such as return_sequences,
        return_state, stateful, unroll...
        """
        super().__init__()
        self.units = units
        self.final_states = None
        self.init_state = None
        if isinstance(cell, str):
            if cell.upper() == "GRU":
                cell = keras.layers.GRUCell
            elif cell.upper() == "LSTM":
                cell = keras.layers.LSTMCell
            elif cell.upper() == "RNN":
                cell = keras.layers.SimpleRNNCell
            else:
                raise Exception("Only GRU, LSTM and RNN are supported as cells.")
        self._cell = cell if hasattr(cell, "units") else cell(units=units)
        self.rnn = keras.layers.RNN(cell=self._cell, *args, **kwargs)

        # single cond
        self.cond_to_init_state_dense_1 = keras.layers.Dense(units=self.units)

        # multi cond
        max_num_conditions = 10
        self.multi_cond_to_init_state_dense = []
        for _ in range(max_num_conditions):
            self.multi_cond_to_init_state_dense.append(
                keras.layers.Dense(units=self.units)
            )
        self.multi_cond_p = keras.layers.Dense(1, activation=None, use_bias=True)

    def _standardize_condition(self, initial_cond):
        initial_cond_shape = initial_cond.shape
        if len(initial_cond_shape) == 2:
            initial_cond = tf.expand_dims(initial_cond, axis=0)
        first_cond_dim = initial_cond.shape[0]
        if isinstance(self._cell, keras.layers.LSTMCell):
            if first_cond_dim == 1:
                initial_cond = tf.tile(initial_cond, [2, 1, 1])
            elif first_cond_dim != 2:
                raise Exception(
                    "Initial cond should have shape: [2, batch_size, hidden_size]\n"
                    "or [batch_size, hidden_size]. Shapes do not match.",
                    initial_cond_shape,
                )
        elif isinstance(self._cell, keras.layers.GRUCell) or isinstance(
            self._cell, keras.layers.SimpleRNNCell
        ):
            if first_cond_dim != 1:
                raise Exception(
                    "Initial cond should have shape: [1, batch_size, hidden_size]\n"
                    "or [batch_size, hidden_size]. Shapes do not match.",
                    initial_cond_shape,
                )
        else:
            raise Exception("Only GRU, LSTM and RNN are supported as cells.")
        return initial_cond

    def __call__(self, inputs, *args, **kwargs):
        """
        :param inputs: List of n elements:
                    - [0] 3-D Tensor with shape [batch_size, time_steps, input_dim]. The inputs.
                    - [1:] list of tensors with shape [batch_size, cond_dim]. The conditions.
        In the case of a list, the tensors can have a different cond_dim.
        :return: outputs, states or outputs (if return_state=False)
        """
        assert isinstance(inputs, list) and len(inputs) >= 2
        x = inputs[0]
        cond = inputs[1:]
        if len(cond) > 1:  # multiple conditions.
            init_state_list = []
            for ii, c in enumerate(cond):
                init_state_list.append(
                    self.multi_cond_to_init_state_dense[ii](
                        self._standardize_condition(c)
                    )
                )
            multi_cond_state = self.multi_cond_p(tf.stack(init_state_list, axis=-1))
            multi_cond_state = tf.squeeze(multi_cond_state, axis=-1)
            self.init_state = tf.unstack(multi_cond_state, axis=0)
        else:
            cond = self._standardize_condition(cond[0])
            if cond is not None:
                self.init_state = self.cond_to_init_state_dense_1(cond)
                self.init_state = tf.unstack(self.init_state, axis=0)
        out = self.rnn(x, initial_state=self.init_state, *args, **kwargs)
        if self.rnn.return_state:
            outputs, h, c = out
            final_states = tf.stack([h, c])
            return outputs, final_states

        return out


Using TensorFlow backend.


In [11]:
#plots.py

import matplotlib.pyplot as plt
import pandas as pd


def history_plot(history, loss_units=None):
    epochs = len(next(iter(history.history.values())))
    ylabel = "loss" if loss_units is None else f"loss ({loss_units})"
    xlabel = "epoch"
    multiline_plot(
        history.history,
        range(1, epochs + 1),
        ticks_from_x=True,
        ylabel=ylabel,
        xlabel=xlabel,
    )


def plot_learning_rate_history(history):
    """
    Uses the history from a model trained with a LearningRateScheduler and plots a chart
    to help pick the optimal learning rate.
    """
    plt.semilogx(history.history["lr"], history.history["loss"])
    plt.axis([1e-8, 1e-4, 0, 30])


def pandas_multiline_plot_from_data(
    data, chart_index=None, figsize=(19, 6),
):
    """
    Takes a passed in dictionary of data names and data, and plots the lines side by side,
    using the pandas plot functionality. The x axis can be configured by passing in a pandas
    index in the chart_index parameter.
    """
    to_plot = pd.DataFrame(data)
    if not chart_index is None:
        data_len = len(next(iter(data.values())))
        to_plot.index = chart_index[0:data_len]

    to_plot.plot(figsize=figsize)


def pandas_multiline_date_plot(
    data, start_date, freq="5min", date_filter=None, figsize=(19, 6),
):
    """
    Takes a passed in dictionary of data names and data, and plots the lines side by side,
    using the pandas plot functionality. The x axis is formed by creating a pandas datetime
    index starting from the parameter start_date. A date filter can be specified to plot
    only a portion of the data by indexing the pandas index (this should be a string or a
    slice).
    """
    to_plot = pd.DataFrame(data)
    data_len = len(next(iter(data.values())))

    date_index = make_datetime_index(start_date, freq=freq, periods=data_len)
    to_plot.index = date_index

    if not date_filter is None:
        to_plot = to_plot.loc[date_filter]

    to_plot.plot(figsize=figsize)


def pandas_2line_plot_from_data(
    data1,
    data2,
    data1_name="Data 1",
    data2_name="Data 2",
    chart_index=None,
    figsize=(19, 6),
):
    """
    Takes 2 passed in iterables of data, and plots them side by side, using the pandas
    plot functionality. The x axis can be configured by passing in a pandas index in the
    chart_index parameter.
    """
    data = {data1_name: data1, data2_name: data2}
    pandas_multiline_plot_from_data(data, chart_index=chart_index, figsize=figsize)


def twoline_plot(
    data1,
    data2,
    data1_name="Data 1",
    data2_name="Data 2",
    x_axis=None,
    figsize=(19, 6),
):
    if x_axis is None:
        x = range(len(data1))
        x_min = 0
        x_max = len(x)
    else:
        x = x_axis[0 : len(data1)]
        x_min = min(x)
        x_max = max(x)

    plt.figure(figsize=figsize)
    plt.xlim(x_min, x_max)
    plt.plot(x, data1, label=data1_name)
    plt.plot(x, data2, label=data2_name)
    plt.legend()
    plt.show()


def multiline_plot(
    data,
    x_axis=None,
    figsize=(19, 6),
    ticks_from_x=False,
    xlabel=None,
    ylabel=None,
    title=None,
):
    data_len = len(next(iter(data.values())))
    if x_axis is None:
        x = range(data_len)
        x_min = 0
        x_max = len(x) - 1
    else:
        x = x_axis[0:data_len]
        x_min = min(x)
        x_max = max(x)

    plt.figure(figsize=figsize)
    plt.xlim(x_min, x_max)
    if x_axis is None or ticks_from_x:
        plt.xticks(range(1, data_len))

    for key, value in data.items():
        plt.plot(x, value, label=key)

    if not xlabel is None:
        plt.xlabel(xlabel)
    if not ylabel is None:
        plt.ylabel(ylabel)
    if not title is None:
        plt.title(title)

    plt.legend()
    plt.show()


def multiline_date_plot(
    data, date_start, figsize=(19, 6), freq="5min",
):
    data_len = len(next(iter(data.values())))
    x_axis = make_daterange(date_start, periods=data_len, freq=freq)
    multiline_plot(data, x_axis, figsize)


In [12]:
# data set tf extra

import tensorflow as tf


def windowed_dataset(
    series, window_size, prediction_size, batch_size, pred_column=0, shuffle=True
):
    """
    Creates a windowed dataset using tf datasets
    """
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + prediction_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + prediction_size + 1))
    if shuffle:
        shuffle_buffer = 1000
        ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(
        lambda w: (
            tf.reshape(w[:window_size], (window_size, series.shape[1])),
            tf.reshape(w[-prediction_size:, pred_column], (prediction_size,)),
        )
    )
    return ds.batch(batch_size).prefetch(1)


In [13]:
# Some global parameters
batch_size = 16
lookback = 36
n_outputs = 12
n_features = 5
EPOCHS = 1

room_names = ["babbage", "babyfoot", "jacquard"]
partitions = list(range(3))

In [14]:
def simple_lstm_compile(model):
    model.compile(loss="mae", optimizer="adam")
    
def simple_lstm_finetune_compile(model):
    import tensorflow.keras as keras
    opt = keras.optimizers.Adam(1e-5)
    
    model.compile(loss="mae", optimizer=opt)
    
def model_simple_lstm(input_shape=(lookback, n_features), n_outputs=12, loss="mae"):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(16, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(16, input_shape=input_shape))
    model.add(keras.layers.Dense(n_outputs))
    # optimizer = keras.optimizers.Adam()
    opt = keras.optimizers.Adam(0.0001)
    model.compile(loss=loss, optimizer=opt)
    return model

def model_simple_lstm3(input_shape=(lookback, n_features), n_outputs=12, loss="mae"):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(16, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(16, input_shape=input_shape))
    model.add(keras.layers.Dense(10))
    model.add(keras.layers.Dense(n_outputs))
    # optimizer = keras.optimizers.Adam()
    opt = keras.optimizers.Adam(0.0001)
    model.compile(loss=loss, optimizer=opt)
    return model

def model_simple_lstm2(input_shape=(lookback, n_features), n_outputs=12, loss="mae"):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(32, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(32, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(16, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(16, input_shape=input_shape))
    model.add(keras.layers.Dense(n_outputs))
    # optimizer = keras.optimizers.Adam()
    opt = keras.optimizers.Adam(0.00005)
    model.compile(loss=loss, optimizer=opt)
    return model

def basic_1dconv(input_shape=(lookback, n_features), n_outputs=12):
    from tensorflow.keras.layers import Conv1D, Dense, Flatten, MaxPooling1D
    from tensorflow.keras.models import Sequential

    activation = "relu"

    model = Sequential()
    model.add(
        Conv1D(
            filters=32,
            kernel_size=3,
            activation=activation,
            input_shape=input_shape,
            name="layer1",
        ),
    )
    model.add(Conv1D(filters=32, kernel_size=3, activation=activation, name="layer2"))
    model.add(MaxPooling1D(pool_size=2, name="layer3"))
    model.add(Conv1D(filters=16, kernel_size=3, activation=activation, name="layer4"))
    model.add(MaxPooling1D(pool_size=2, name="layer5"))
    model.add(Flatten(name="layer6"))
    model.add(Dense(100, activation=activation, name="layer7"))
    model.add(Dense(n_outputs, name="layer8"))
    opt = keras.optimizers.Adam(0.00005)
    model.compile(loss="mae", optimizer=opt)
    return model

In [15]:
import pandas as pd
df = pd.read_csv(
    "buildingDB.csv", 
    # parse_dates=["time"],
    index_col=["time"],
    error_bad_lines=False
    )
df_main = df.drop(["presence"], axis=1)
# Careful with the following assumption. It worked for Jan, March
# to set missing sensor values to zero, but is it still good for march-aug?
df_main = df_main.fillna(df_main.min(),) 

def filter_room(df, room):
    return df.loc[df.room == room].drop(["room"], axis=1)

In [16]:
class QarnotDataReader:
    def __init__(self, gen_val=False, dataset=df_main, split_date='2020-03-08'):

        df = dataset.copy()
        self.split_date = split_date

        split_index = find_split_index(df, self.split_date)
        means = np.mean(df.iloc[:split_index, 1:])
        stds = np.std(df.iloc[:split_index, 1:])

        df.iloc[:, 1:] = (df.iloc[:, 1:] - means) / stds

        self.names = room_names
        self.df = df
        self.scalers = {}
        self.gen_val = gen_val
        self.lookback = lookback
        self.n_outputs = n_outputs
        self.batch_size = batch_size


    def get_number_of_partitions(self):
        return len(self.names)

    def get_partitions(self):
        return self.names

    def get_partition_name(self, pid):
        return self.names[pid]

    def get_data_for_partition(self, partition):
        partition_name = self.get_partition_name(partition)

        return self.get_windowed_data_for_partition(
            partition_name, self.lookback, self.n_outputs, self.batch_size
        )
    
    def get_general_test_set(self):
        
        (
            _,
            _1,
            test_x,
            _2,
            test_y,
            _3,
        ) = prepare_inputs_by_partition(
            self.df, "room", self.split_date, output_col=0, lookback=self.lookback, 
            num_predictions=self.n_outputs,
        )
        
        return test_x, test_y


    def get_windowed_data_for_partition(
        self, partition, lookback, n_outputs, batch_size
    ):

        data = self.df[self.df.room == partition].drop(["room"], axis=1)
        split_index = find_split_index(data, self.split_date)
        scaler, norm_data = normalize_dataset(data, split_index)

        self.scalers[partition] = scaler

        train_data = windowed_dataset(
            norm_data[0:split_index], lookback, n_outputs, batch_size
        )

        if self.gen_val:
            return get_general_test_set()
        else:
            val_data = windowed_dataset(
                norm_data[split_index:], lookback, n_outputs, batch_size
            )

        return (train_data, val_data)


qarnot_provider = lambda: QarnotDataReader()
qarnot_provider_genval = lambda: QarnotDataReader(gen_val=True)

In [19]:
def sort_tuple(name):
    parts = str(name).split("_")
    parts = list([int(p) if p.isdigit() else p for p in parts])
    return (parts[0], parts[0] if len(parts) < 2 else parts[1], name)


def sorted_names(name_list):
    return list(map(lambda x: x[-1], sorted(map(sort_tuple, name_list))))


def line_chart(ds: pd.DataFrame, round_column: str = "loss", labels=None):
    margins = 0.0

    # fig = plt.figure(figsize=(6, 4))  # a new figure window, figsize goes here
    fig = plt.figure(figsize=(8, 6))  # a new figure window, figsize goes here
    ax = fig.add_subplot(1, 1, 1)  # specify (nrows, ncols, axnum)

    dsg = ds.groupby("partition")
    legends = []
    i = 0
    for name, group in dsg:
        label = labels[i] if labels else name
        legends.append(label)
        ax.plot(group["epoch"], group[round_column])
        i += 1

    # decorate then sort
    #legends = sorted_names(legends)

    if len(legends) > 4:
        ax.legend(legends[0 : min(12, len(legends))], fancybox=True, framealpha=0.5)
    else:
        ax.legend(legends)
    ax.set_xlabel("Training epoch")
    ax.set_ylabel(round_column)
    ax.set_title(f"Metric {round_column}")

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

    plt.gca().set_xticks(list(range(ds["epoch"].min(), ds["epoch"].max() + 1)))

    # if tight:
    #     plt.tight_layout(h_pad=3)

    plt.margins(margins)
    
def line_chart_2(dsets: list, partition: int, round_column: str, labels: list = None):
    dsets2 = []
    for i, s in enumerate(dsets):
        s2 = s[s.partition == partition].copy()
        s2["partition"] =  s2["partition"].apply(lambda x: f"{x}_exp{i + 1}")
        dsets2.append(s2)
        
    all_data = pd.concat(dsets2)
    line_chart(ds=all_data, round_column=round_column, labels=labels)

In [20]:
def history_to_df(all_history):
    data = {"epoch": [], "partition": []}

    epochs = {}
    for round, round_his in enumerate(all_history):
        for idx, his in enumerate(round_his.items()):
            # print(idx, his)
            partition = his[0]
            history = his[1]
            keys = list(history.keys())
            for key in keys:
                if key not in data:
                    data[key] = []

            for i in range(len(history[keys[0]])):
                epoch = epochs.get(partition, 1)
                data["partition"].append(partition)
                data["epoch"].append(int(epoch))
                for key in keys:
                    data[key].append(history[key][i])
                epochs[partition] = epoch + 1

    return pd.DataFrame(data=data)

In [21]:
def transfer_all(model, weights):
    model.set_weights(weights)


def fed_avg(weights):
    new_weights = list()
    for weights_list_tuple in zip(*weights):
        new_weights.append(
            np.array([np.array(w).mean(axis=0) for w in zip(*weights_list_tuple)])
        )

    return new_weights


def make_weight_averager(base_weights=None):
    buffer = []
    partition_meta = []

    def partition_trained(base_weights, weights, metadata={}):
        partition_meta.append(metadata)
        buffer.append(weights)
        return base_weights

    def round_end():
        new_weights = fed_avg(buffer)
        buffer.clear()
        partition_meta.clear()
        return new_weights

    sn = SimpleNamespace()
    sn.partition_trained = partition_trained
    sn.round_end = round_end
    return sn


# Federated learning train algorithm
def make_train_algo(partitions, base_weights, data_provider, model_algo):
    data = data_provider
    models = {}

    for p in partitions:
        models[p] = model_algo()
        models[p].set_weights(base_weights)

    def train_partition(partition, current_weights):
        gen_scores = []
        gen_x, gen_y = data.get_general_test_set()
        
        class GenvalCallback(tf.keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs=None):
                score = self.model.evaluate(gen_x, gen_y, verbose = 0)
                gen_scores.append(score)
                if logs:
                    logs["gen_loss"] = score
                
        model = models[partition]
        model.set_weights(current_weights)
        train_data, val_data = data.get_data_for_partition(partition)
        history = model.fit(
            train_data, epochs=EPOCHS, verbose=1, validation_data=val_data, callbacks=[GenvalCallback()]
        )
        results = history.history.copy()
        results["gen_loss"] = gen_scores
        
        return results, model.get_weights()

    return train_partition

        
        
def make_train_algo_pers4(partitions, base_weights, data_provider, model_algo):
    data = data_provider
    local_models = {}
    LAYERS = 2

    # for p in partitions:
    #   models[p] = (model_algo())
    #   models[p].set_weights(base_weights)

    for p in partitions:
        local_models[p] = model_algo()
        local_models[p].set_weights(base_weights)

    def train_partition(partition, current_weights):
        #    model = models[partition]
        #    model.set_weights(current_weights)
        gen_scores = []
        gen_x, gen_y = data.get_general_test_set()
        
        class GenvalCallback(tf.keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs=None):
                score = self.model.evaluate(gen_x, gen_y, verbose = 0)
                gen_scores.append(score)
                if logs:
                    logs["gen_loss"] = score
                
        local_model = local_models[partition]
        local_weights = local_model.get_weights()
        local_weights[0:LAYERS] = current_weights[0:LAYERS]
        local_model.set_weights(local_weights)

        train_data, val_data = data.get_data_for_partition(partition)

        history = local_model.fit(
            train_data, epochs=EPOCHS, verbose=1, validation_data=val_data, callbacks=[GenvalCallback()]
        )

        local_weights = local_model.get_weights()
        
        results = history.history.copy()
        results["gen_loss"] = gen_scores
        
        return results, local_model.get_weights()

    return train_partition


def train_partition(partition, data, get_model, epochs=5, verbose=0):
    gen_scores = []
    gen_x, gen_y = data.get_general_test_set()

    class GenvalCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            score = self.model.evaluate(gen_x, gen_y, verbose = 0)
            gen_scores.append(score)
            if logs:
                logs["gen_loss"] = score
            
            
    model = get_model()

    train_data, val_data = data.get_data_for_partition(partition)
    history = model.fit(train_data, epochs=epochs, verbose=1, validation_data=val_data, callbacks=[GenvalCallback()])
    results = history.history.copy()
    results["gen_loss"] = gen_scores
    
    return results, model


def make_trainer(partition_chooser, base_weights, train_algo, averager):
    run_data = {"rounds": 0, "weights": base_weights}
    all_histories = []

    def train_round():
        histories = {}

        for p in partition_chooser():
            # print("Train partition", p)
            current_weights = run_data["weights"]
            history, weights = train_algo(p, current_weights)
            # Perform any required averaging when each partition is trained (async fedavg for example)
            run_data["weights"] = averager.partition_trained(current_weights, weights)
            histories[p] = history

        # Perform any required averaging for the end of a round (fedavg for example)
        run_data["weights"] = averager.round_end()
        all_histories.append(histories)
        run_data["rounds"] = run_data["rounds"] + 1

    def run_rounds(num_rounds):
        for r in range(num_rounds):
            print("Training round", r)
            train_round()

        return all_histories

    sn = SimpleNamespace()
    sn.run_rounds = run_rounds
    sn.rounds_run = lambda: run_data["rounds"] # number of rounds that were run
    sn.global_weights = lambda: run_data["weights"]
    return sn

In [22]:
def all_partitions(all_partitions):
    def make_partitions():
        return all_partitions

    return make_partitions


def build_trainer(
    model_algo=model_simple_lstm,
    data_provider=qarnot_provider,
    train_algo_provider=make_train_algo,
    averager=make_weight_averager,
    partition_algo=all_partitions,
    partition_selection=None,
    starting_weights=None,
):
    data = data_provider()
    
    if starting_weights is None:
        model = model_algo()
        base_weights = model.get_weights()
    else:
        base_weights = starting_weights
            
    if partition_selection is None:
        partitions = range(get_number_of_partitions())
    else:
        partitions = partition_selection

    train_algo = train_algo_provider(partitions, base_weights, data, model_algo)
    avg_algo = averager(base_weights)
    partition_chooser = partition_algo(partitions)

    return make_trainer(partition_chooser, base_weights, train_algo, avg_algo)

In [17]:
model = model_simple_lstm()
data = QarnotDataReader(dataset=df_main, split_date='2020-03-08')

train_data, val_data = data.get_data_for_partition(0)
# Quick test to see if everything is working
history = model.fit(train_data, validation_data=val_data, epochs=1)

In [23]:
def pd_date(date):
    return date.strftime("%Y-%m-%d")

def str_to_pddate(date_string):
    return datetime.datetime.strptime(date_string, '%Y-%m-%d')

In [26]:
# Test all the above code by training a model on a slice of data
rounds = 1
model_provider = model_simple_lstm
dataset=df_main[df_main.index<"2020-01-10"] # slice of data to use
split_date= "2020-01-07" # date of train/test split
partition_selection = partitions # or [0,2] for example

data_provider = lambda: QarnotDataReader(dataset=dataset, split_date=split_date)

trainer = build_trainer(
    model_algo=model_provider,
    partition_selection=partition_selection,
    train_algo_provider=make_train_algo, # federated learning
    data_provider=data_provider,
)
all_history = trainer.run_rounds(rounds)
trained_weights = trainer.global_weights

Training round 0
106/106 [==============================] - 4s 36ms/step - loss: 0.4903 - val_loss: 0.6178


In [27]:
trainweek = 2
validateweek=1

df_main.index = pd.to_datetime(df_main.index)
start_date = df_main.index.min().to_pydatetime()
end_date = df_main.index.max().to_pydatetime() - datetime.timedelta(7*(trainweek+validateweek))
rolling_dates = make_daterange(start_date = start_date, end_date = end_date, freq="1w")

In [ ]:
EPOCHS = 1 # Number of epochs to train each model before fedavg
rounds = 1 # Number of rounds of epochs then fedavg to run
model_provider = model_simple_lstm
partition_selection = partitions # or [0,2] for example

test_model = model_provider()
running_weights = test_model.get_weights()

# Create a dataframe that stores all results
base_df = pd.DataFrame(columns=["epoch", "partition", "loss", "val_loss", "gen_loss", "date"])

for index, date in enumerate(rolling_dates[(trainweek+validateweek):]):
    
    print("Training for date", date)
    
    ## Take 3 weeks, train on 2, validate on 1
    start_date = date - datetime.timedelta(7*(trainweek+validateweek))
    split_date = date - datetime.timedelta(7*validateweek)
    # import pdb; pdb.set_trace()
    dataset=df_main[df_main.index > pd_date(start_date)]  
    dataset = dataset[dataset.index < pd_date(date)] # slice of data to use

    data_provider = lambda: QarnotDataReader(dataset=dataset, split_date=pd_date(split_date))

    trainer = build_trainer(
        model_algo=model_provider,
        partition_selection=partition_selection, # all_partitions
        train_algo_provider=make_train_algo, # federated learning
        data_provider=data_provider,
        starting_weights = running_weights
    )
    
    # Accumulate results in a dataframe
    all_history = trainer.run_rounds(rounds)
    df_next = history_to_df(all_history)
    df_next["date"] = date
    base_df = base_df.append(df_next)
    base_df.reset_index(inplace=True, drop=True)
    
    running_weights = trainer.global_weights()
  
    print()
    
    # You can perform another validation here by loading weights into the model
    # test_model = model_provider()
    # test_model.set_weights(trained_weights)

In [36]:
base_df.to_csv("fedavg_results.csv", index=False)

In [ ]:
# WIP Train the individual partitions with no federation. Note that to compare with
# the experiments including federation above, you need to make sure total epochs, learning rates
# etc all match up for a fair comparison

EPOCHS=1
model_provider = model_simple_lstm

part_df = pd.DataFrame(columns=["epoch", "partition", "loss", "val_loss", "gen_loss", "date"])

model_providers = { 0: model_provider, 1: model_provider, 2: model_provider }

for index, date in enumerate(rolling_dates[(trainweek+validateweek):]):
    
    print("Training for date", date)
    
    ## Take 3 weeks, train on 2, validate on 1
    start_date = date - datetime.timedelta(7*(trainweek+validateweek))
    split_date = date - datetime.timedelta(7*validateweek)
    # import pdb; pdb.set_trace()
    dataset=df_main[df_main.index > pd_date(start_date)]  
    dataset = dataset[dataset.index < pd_date(date)] # slice of data to use

    data_provider = lambda: QarnotDataReader(dataset=dataset, split_date=split_date)

    histories = []
    for i in partitions:
        his_i, model_i = train_partition(i, data_provider(), model_providers[i], epochs=EPOCHS)
        model_providers[i] = lambda: model_i
        
        df_i = history_to_df([{i:his_i}])
        histories.append(df_i)

    df_next = pd.concat(histories)
    df_next["date"] = date
    part_df = part_df.append(df_next)
    part_df.reset_index(inplace=True, drop=True)

In [ ]:
part_df